1. Create a heat map that displays the humidity for every city from the part I of the homework.

2. Narrow down the DataFrame to find your ideal weather condition. For example: (A max temperature lower than 80 degrees but higher than 70., Wind speed less than 10 mph., Zero cloudiness.) Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

3. Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

4. Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [50]:
#Import dependencies
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests

In [51]:
#Import csv
city_data = "Resources/citydata.csv"

# Read City Data File and store into Pandas DataFrames
city_data = pd.read_csv(city_data)

In [52]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [53]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
humidity = city_data["Humidity"].astype(float)

In [54]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
#Narrow down data frame
# Temp between 60-75, Wind speed less than 10, Clouds less than 20, Humidity less than 55
ideal_subset=city_data.loc[(city_data['Temperature'] >= 60) & (city_data['Temperature'] <= 72) & 
                           (city_data['Wind Speed'] <= 10) & (city_data['Cloudiness'] <= 0) & (city_data['Humidity'] < 55)]

ideal_subset

,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
155,bulawayo,-20.15,28.58,68.29,34.0,0.0,6.78,ZW,1.603572e+09
258,turayf,31.67,38.66,62.60,25.0,0.0,8.05,SA,1.603572e+09
314,nizwa,22.93,57.53,67.78,25.0,0.0,5.59,OM,1.603572e+09
317,katra,32.98,74.95,60.40,36.0,0.0,9.62,IN,1.603572e+09
515,abha,18.22,42.51,64.40,16.0,0.0,4.70,SA,1.603572e+09
535,faya,18.39,42.45,64.40,16.0,0.0,4.70,SA,1.603572e+09
590,hun,29.13,15.95,63.07,41.0,0.0,3.89,LY,1.603572e+09
593,buraydah,26.33,43.98,68.00,28.0,0.0,5.50,SA,1.603572e+09
646,marzuq,14.40,46.47,64.45,26.0,0.0,0.74,YE,1.603571e+09


In [57]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
#Search for nearest hotels within 5,000 m

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_subset.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['latitude'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# Convert hotels to list
hotel_list = census_data_complete["Bank Count"].tolist()

In [ ]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel location: {hotel}" for hotel in hotel_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

In [ ]:
# Create a combined map with hotels and heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig